In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 원유 가격 데이터 전처리

In [2]:
# 2012~2021년 각 월별 원유 데이터
oil_day = pd.read_csv('./data/국제_원유가격20120101_20211231.csv', encoding='utf-8')
oil_day

,기간,Dubai,Brent,WTI
0,12년01월03일,105.91,112.13,102.96
1,12년01월04일,108.49,113.70,103.22
2,12년01월05일,110.23,112.74,101.81
3,12년01월06일,109.92,113.06,101.56
4,12년01월09일,110.50,112.45,101.31
...,...,...,...,...
2576,21년12월27일,NaN,78.60,75.57
2577,21년12월28일,76.47,78.94,75.98
2578,21년12월29일,76.91,79.23,76.56
2579,21년12월30일,77.24,79.32,76.99


In [3]:
oil_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2581 entries, 0 to 2580
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기간      2581 non-null   object 
 1   Dubai   2503 non-null   float64
 2   Brent   2573 non-null   float64
 3   WTI     2514 non-null   float64
dtypes: float64(3), object(1)
memory usage: 80.8+ KB


In [4]:
oil_day.rename(columns={'기간':'date'}, inplace=True)
oil_day

,date,Dubai,Brent,WTI
0,12년01월03일,105.91,112.13,102.96
1,12년01월04일,108.49,113.70,103.22
2,12년01월05일,110.23,112.74,101.81
3,12년01월06일,109.92,113.06,101.56
4,12년01월09일,110.50,112.45,101.31
...,...,...,...,...
2576,21년12월27일,NaN,78.60,75.57
2577,21년12월28일,76.47,78.94,75.98
2578,21년12월29일,76.91,79.23,76.56
2579,21년12월30일,77.24,79.32,76.99


In [5]:
# 원유가격 데이터 결측치 확인
oil_day.isnull().sum()

date      0
Dubai    78
Brent     8
WTI      67
dtype: int64

In [6]:
# 결측치 발생한 지점을 위아래 값의 평균으로 대체
# interpolate함수 : 결측치를 앞뒤값의 평균으로 처리해주는 함수. 시계열데이터 결측치처리에 효과적.
oil_day.interpolate(inplace=True)
oil_day.isnull().sum()

date     0
Dubai    0
Brent    0
WTI      0
dtype: int64

In [7]:
oil_day['date'] = oil_day['date'].str.replace('년','-')
oil_day['date'] = oil_day['date'].str.replace('월','-')
oil_day['date'] = oil_day['date'].str.replace('일','')
oil_day

,date,Dubai,Brent,WTI
0,12-01-03,105.91,112.13,102.96
1,12-01-04,108.49,113.70,103.22
2,12-01-05,110.23,112.74,101.81
3,12-01-06,109.92,113.06,101.56
4,12-01-09,110.50,112.45,101.31
...,...,...,...,...
2576,21-12-27,75.42,78.60,75.57
2577,21-12-28,76.47,78.94,75.98
2578,21-12-29,76.91,79.23,76.56
2579,21-12-30,77.24,79.32,76.99


In [8]:
oil_day['date'] = '20' + oil_day['date'].astype(str)
oil_day

,date,Dubai,Brent,WTI
0,2012-01-03,105.91,112.13,102.96
1,2012-01-04,108.49,113.70,103.22
2,2012-01-05,110.23,112.74,101.81
3,2012-01-06,109.92,113.06,101.56
4,2012-01-09,110.50,112.45,101.31
...,...,...,...,...
2576,2021-12-27,75.42,78.60,75.57
2577,2021-12-28,76.47,78.94,75.98
2578,2021-12-29,76.91,79.23,76.56
2579,2021-12-30,77.24,79.32,76.99


In [9]:
# type을 str에서 datetime으로 바꿈
oil_day['date'] = pd.to_datetime(oil_day['date'])

In [10]:
oil_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2581 entries, 0 to 2580
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2581 non-null   datetime64[ns]
 1   Dubai   2581 non-null   float64       
 2   Brent   2581 non-null   float64       
 3   WTI     2581 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 80.8 KB


In [11]:
oil_day['oil_avg']=( oil_day['Dubai']+oil_day['Brent']+oil_day['WTI'] ) /3
oil_day

,date,Dubai,Brent,WTI,oil_avg
0,2012-01-03,105.91,112.13,102.96,107.000000
1,2012-01-04,108.49,113.70,103.22,108.470000
2,2012-01-05,110.23,112.74,101.81,108.260000
3,2012-01-06,109.92,113.06,101.56,108.180000
4,2012-01-09,110.50,112.45,101.31,108.086667
...,...,...,...,...,...
2576,2021-12-27,75.42,78.60,75.57,76.530000
2577,2021-12-28,76.47,78.94,75.98,77.130000
2578,2021-12-29,76.91,79.23,76.56,77.566667
2579,2021-12-30,77.24,79.32,76.99,77.850000


In [12]:
# 일별 원유 가격 데이터 파일 저장
oil_day.to_csv('./pre_data/일별_원유가격.csv',encoding='utf-8')

In [13]:
# 월별 원유 가격 데이터 만들기
oil_month = oil_day.resample('M',on='date').mean()
oil_month

,Dubai,Brent,WTI,oil_avg
date,,,,
2012-01-31,109.456667,111.452381,100.289286,107.066111
2012-02-29,116.180952,119.060000,102.370952,112.537302
2012-03-31,122.487273,124.544545,106.205000,117.745606
2012-04-30,117.338500,120.486500,103.346000,113.723667
2012-05-31,107.719348,110.288696,94.546087,104.184710
...,...,...,...,...
2021-08-31,69.518864,70.513636,67.711364,69.247955
2021-09-30,72.628182,74.876364,71.420909,72.975152
2021-10-31,81.610952,83.746190,81.222857,82.193333


In [14]:
oil_month = oil_month.reset_index()
oil_month

,date,Dubai,Brent,WTI,oil_avg
0,2012-01-31,109.456667,111.452381,100.289286,107.066111
1,2012-02-29,116.180952,119.060000,102.370952,112.537302
2,2012-03-31,122.487273,124.544545,106.205000,117.745606
3,2012-04-30,117.338500,120.486500,103.346000,113.723667
4,2012-05-31,107.719348,110.288696,94.546087,104.184710
...,...,...,...,...,...
115,2021-08-31,69.518864,70.513636,67.711364,69.247955
116,2021-09-30,72.628182,74.876364,71.420909,72.975152
117,2021-10-31,81.610952,83.746190,81.222857,82.193333
118,2021-11-30,80.317500,80.849545,78.409091,79.858712


In [15]:
oil_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     120 non-null    datetime64[ns]
 1   Dubai    120 non-null    float64       
 2   Brent    120 non-null    float64       
 3   WTI      120 non-null    float64       
 4   oil_avg  120 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 4.8 KB


In [16]:
# 월별 원유 가격 데이터 파일 저장
oil_month.to_csv('./pre_data/월별_원유가격.csv',encoding='utf-8')

## 종계입식 현황 데이터 전처리 (단위: 천)

In [18]:
breeding =pd.read_csv('./data/2012_2021종계입식현황.csv', encoding='utf-8')
breeding

,date,count
0,2012-01-31,356
1,2012-02-29,672
2,2012-03-31,870
3,2012-04-30,718
4,2012-05-31,496
...,...,...
115,2021-08-31,670
116,2021-09-30,479
117,2021-10-31,591
118,2021-11-30,652


In [19]:
breeding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    120 non-null    object
 1   count   120 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.0+ KB


In [20]:
# type을 str에서 datetime으로 바꿈
breeding['date'] = pd.to_datetime(breeding['date'])

In [21]:
breeding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    120 non-null    datetime64[ns]
 1   count   120 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 2.0 KB


In [22]:
# 월별 종계입식 데이터 파일 저장
breeding.to_csv('./pre_data/월별_종계입식현황.csv',encoding='utf-8')